# Imports

In [399]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

# Data import & Cleaning

In [400]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
spray_df = pd.read_csv('../data/spray.csv')
weather_df = pd.read_csv('../data/weather.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

***Weather Initial Analysis***

In [401]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [402]:
weather_df.columns.tolist()

['Station',
 'Date',
 'Tmax',
 'Tmin',
 'Tavg',
 'Depart',
 'DewPoint',
 'WetBulb',
 'Heat',
 'Cool',
 'Sunrise',
 'Sunset',
 'CodeSum',
 'Depth',
 'Water1',
 'SnowFall',
 'PrecipTotal',
 'StnPressure',
 'SeaLevel',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed']

In [403]:
# Check for missing values

weather_df.isnull().sum().sort_values(ascending= False)

Station        0
Date           0
ResultDir      0
ResultSpeed    0
SeaLevel       0
StnPressure    0
PrecipTotal    0
SnowFall       0
Water1         0
Depth          0
CodeSum        0
Sunset         0
Sunrise        0
Cool           0
Heat           0
WetBulb        0
DewPoint       0
Depart         0
Tavg           0
Tmin           0
Tmax           0
AvgSpeed       0
dtype: int64

In [404]:
weather_df.isnull().sum()[weather_df.isnull().sum() > 0]

Series([], dtype: int64)

In [405]:
# Checking for duplicated values

weather_df.duplicated().sum()

0

In [406]:
weather_df.duplicated().sum()[weather_df.duplicated().sum() > 0]

array([], dtype=int64)

In [407]:
weather_df.shape

(2944, 22)

In [408]:
# Check for 'M' or missing values in certain columns

weather_df.isin(['M']).sum()[weather_df.isin(['M']).sum() > 0].sort_values(ascending = False)

Water1         2944
Depart         1472
Depth          1472
SnowFall       1472
Tavg             11
Heat             11
Cool             11
SeaLevel          9
WetBulb           4
StnPressure       4
AvgSpeed          3
PrecipTotal       2
dtype: int64

In [409]:
# Check for '-' in Sunrise/Sunset columns

weather_df.isin(['-']).sum()[weather_df.isin(['-']).sum() > 0].sort_values(ascending = False)

Sunrise    1472
Sunset     1472
dtype: int64

In [410]:
# Check for 'T' or trace values

weather_df.isin(['  T']).sum()[weather_df.isin(['  T']).sum() > 0].sort_values(ascending = False)

PrecipTotal    318
SnowFall        12
dtype: int64

In [411]:
# Check for '0' values

weather_df.isin(['0']).sum()[weather_df.isin(['0']).sum() > 0].sort_values(ascending = False)

Heat     1870
Depth    1472
dtype: int64

In [412]:
# Check for '0' values that are in float form, aka '0.0'

weather_df.isin(['0.0']).sum()[weather_df.isin(['0.0']).sum() > 0].sort_values(ascending = False)

SnowFall    1459
dtype: int64

***Feature Engineering: imputing / dropping features***

In [413]:
# Drop columns that have high missing values (almost, if not 100%) of 'M' or '0'

weather_df = weather_df.drop(columns = ['Water1', 'Depth', 'SnowFall'])

In [414]:
# Create a function to impute for Tavg, Heat and Cool

def impute_tavg_heat_cool(row):
    # Impute for 'M' in Tavg
    if row['Tavg'] == 'M':
        row['Tavg'] = round((row['Tmax'] - row['Tmin'])/2)
    
    # Impute for 'M' in Heat and Cool
    if row['Cool'] == 'M' or row['Heat'] == 'M':
        hc = 65 - row['Tavg']
        if hc < 0:
            row['Cool'] = hc
            row['Heat'] = 0
        elif hc > 0:
            row['Cool'] = 0
            row['Heat'] = hc
        else:
            row['Cool'] = row['Heat'] = 0
    return row

In [415]:
weather_df = weather_df.apply(impute_tavg_heat_cool, axis = 1)

In [416]:
# Create function to extract 30 year normal temperature from station 1's readings

def extract_normal(row):
    if row['Station'] == 1:
        row['Normal'] = int(row['Tavg']) - int(row['Depart'])
    return row

In [417]:
# Apply 30 year temperature to station 2's reading

def apply_normal(row):
    if row['Station'] == 2:
        row['Normal'] = weather_df[(weather_df['Date'] == row['Date']) & (weather_df['Station'] ==1)]['Normal'].values[0]
    return row

In [418]:
# Apply both function into weather_df

weather_df = weather_df.apply(extract_normal, axis = 1)
weather_df = weather_df.apply(apply_normal, axis = 1)

In [419]:
weather_df.head()

,AvgSpeed,CodeSum,Cool,Date,Depart,DewPoint,Heat,Normal,PrecipTotal,ResultDir,ResultSpeed,SeaLevel,Station,StnPressure,Sunrise,Sunset,Tavg,Tmax,Tmin,WetBulb
0,9.2,,2,2007-05-01,14,51,0,53.0,0.00,27,1.7,29.82,1,29.10,0448,1849,67,83,50,56
1,9.6,,3,2007-05-01,M,51,0,53.0,0.00,25,2.7,29.82,2,29.18,-,-,68,84,52,57
2,13.4,BR,0,2007-05-02,-3,42,14,54.0,0.00,4,13.0,30.09,1,29.38,0447,1850,51,59,42,47
3,13.4,BR HZ,0,2007-05-02,M,42,13,54.0,0.00,2,13.3,30.08,2,29.44,-,-,52,60,43,47
4,11.9,,0,2007-05-03,2,40,9,54.0,0.00,7,11.7,30.12,1,29.39,0446,1851,56,66,46,48


In [420]:
# Rearrange the columns
weather_df = weather_df[['Station',
 'Date',
 'Tmax',
 'Tmin',
 'Tavg',
 'Depart',
 'DewPoint',
 'WetBulb',
 'Heat',
 'Cool',
 'Normal',
 'Sunrise',
 'Sunset',
 'CodeSum',
 'PrecipTotal',
 'StnPressure',
 'SeaLevel',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed']]

In [421]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,53.0,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,53.0,-,-,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,54.0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,54.0,-,-,BR HZ,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,54.0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9


In [422]:
# Checking for the remaining 'M' values over the entire notebook

weather_df.isin(['M']).sum()[weather_df.isin(['M']).sum() > 0]

Depart         1472
WetBulb           4
PrecipTotal       2
StnPressure       4
SeaLevel          9
AvgSpeed          3
dtype: int64

In [423]:
# Impute for remaining columns as listed above: Depart, WetBulb, PrecipTotal,StnPressure, SeaLevel, AvgSpeed

def impute_M(row):
    replace_dict = {}
    for index in row.index:
        if row[index] == 'M':
            replace_dict[index] = 'M'
    
    if replace_dict:
        if'Depart' in replace_dict:
            row['Depart'] = int(row['Tavg']) - int(row['Normal'])
            del replace_dict['Depart']

        for key, value in replace_dict.items():
            #Retrive from station 1 if Station 2 is 'M' and vice versa
            if row['Station'] == 2:
                row[key] = weather_df[(weather_df['Date'] == row['Date']) & (weather_df['Station'] == 1)][key].values[0]
            else:
                row[key] = weather_df[(weather_df['Date'] == row['Date']) & (weather_df['Station'] == 2)][key].values[0]

    return row

In [424]:
weather_df = weather_df.apply(impute_M, axis = 1)

In [425]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,53.0,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,15,51,57,0,3,53.0,-,-,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,54.0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,-2,42,47,13,0,54.0,-,-,BR HZ,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,54.0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9


In [426]:
# Check again if there's any rows with 'M' left out

weather_df.isin(['M']).sum()[weather_df.isin(['M']).sum() > 0].sort_values(ascending = False)

StnPressure    2
dtype: int64

In [427]:
# Lets check these 2 rows of StnPressure

weather_df[weather_df['StnPressure'] == 'M']

# Function didnt work becuase both station 1 & 2 both have 'M' values for StnPressure

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
2410,1,2013-08-10,81,64,73,0,57,63,0,8,73.0,0454,1900,,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,75,2,55,63,0,10,73.0,-,-,,0.00,M,30.07,6.0,6,7.4


In [428]:
# In that case, lets just impute the StnPressure input of the day before

weather_df.at[2410, 'StnPressure'] = weather_df[weather_df['Date'] == '2013-08-09']['StnPressure'].values[0]
weather_df.at[2411, 'StnPressure'] = weather_df[weather_df['Date'] == '2013-08-09']['StnPressure'].values[0]

In [429]:
# Check the 2 rows again

weather_df.loc[[2410, 2411]]

# StnPressure imputed properly

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
2410,1,2013-08-10,81,64,73,0,57,63,0,8,73.0,0454,1900,,0.00,29.28,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,75,2,55,63,0,10,73.0,-,-,,0.00,29.28,30.07,6.0,6,7.4


In [430]:
# Not to deal with '  T' inputs, we will be changing '  T' to 0.01
weather_df['PrecipTotal'] = weather_df['PrecipTotal'].apply(lambda x : 0.01 if '  T' in x else x)

In [431]:
# Check if there are inputs with '  T' remaining

weather_df.isin(['  T']).sum()[weather_df.isin(['  T']).sum() > 0].sort_values(ascending = False)

# No more rows with '  T' remaining

Series([], dtype: int64)

In [432]:
# We will now change the date column from string to datetime

weather_df['Date'] = pd.to_datetime(weather_df['Date'])

In [433]:
# Check if date column is changed to datetime

weather_df.info()

# Changed confirmed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   int64         
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   int64         
 3   Tmin         2944 non-null   int64         
 4   Tavg         2944 non-null   object        
 5   Depart       2944 non-null   object        
 6   DewPoint     2944 non-null   int64         
 7   WetBulb      2944 non-null   object        
 8   Heat         2944 non-null   object        
 9   Cool         2944 non-null   object        
 10  Normal       2944 non-null   float64       
 11  Sunrise      2944 non-null   object        
 12  Sunset       2944 non-null   object        
 13  CodeSum      2944 non-null   object        
 14  PrecipTotal  2944 non-null   object        
 15  StnPressure  2944 non-null   object        
 16  SeaLev

In [434]:
# Impute sunrise / sunset for some rows that marked their sunrise / sunset as '-'

def impute_sun(row):
    if row['Station'] == 2:
        row['Sunrise'] = weather_df[(weather_df['Date'] == row['Date']) & (weather_df['Station'] == 1)]['Sunrise'].values[0]
        row['Sunset'] = weather_df[(weather_df['Date'] == row['Date']) & (weather_df['Station'] == 1)]['Sunset'].values[0]
    return row

In [435]:
# Apply the impute_sun function

weather_df = weather_df.apply(impute_sun, axis = 1)

In [436]:
# Check if the function have imputed correctly

weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,53.0,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,15,51,57,0,3,53.0,0448,1849,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,54.0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,-2,42,47,13,0,54.0,0447,1850,BR HZ,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,54.0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9


In [437]:
# Check if there are any '- that are left out

weather_df.isin(['-']).sum()[weather_df.isin(['-']).sum() > 0].sort_values(ascending = False)

Series([], dtype: int64)

There are various weather phenomeno recorded everyday under 'CodeSum' column. Might need to look into the weather patterns to see if there are some kind of correlation with our target variable.

There are some input like TSRA that could have been a mix of thunderstorm for TS and rain for RA that is not included in the data dictionary provided.

In [438]:
# First to separate the codes in accordance to the weather codenames provided within the data dictionary

w_codenames = ['+FC','FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL',
         'IC', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS',
         'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR',
         'BC', 'BL', 'VC']

weather_df['CodeSum'] = weather_df['CodeSum'].apply(lambda x: ' '. join([t for t in x.split(' ') if t in w_codenames]))

In [439]:
# To also create a new column feature with the number of weather phenomena within a single day 

weather_df['n_weather_per_day'] = weather_df['CodeSum'].apply(lambda x: len(x.split()))

In [440]:
snow = ['SN', 'SG', 'GS', 'PL', 'IC', 'DR', 'BC']
windy = ['SQ', 'DS', 'SS', 'PO', 'BL']
rain = ['TS', 'GR', 'RA', 'DZ', 'SH']
lowvis = ['FG+', 'FG', 'BR', 'HZ']
codesum_others = ['UP', 'VA', 'DU', 'SA', 'FZ', '+FC', 'FC', 'FU', 'PY', 'MI', 'PR', 'BC', 'BL', 'VC' ]

In [441]:
# Make a function to dummify each weather codenames based on their category above

def w_code_dummy_split(row):
    w_codes = row['CodeSum'].split()

    # Check for weather conditions
    if any(code in w_codes for code in snow):
        row['snow'] = 1
    
    if any(code in w_codes for code in windy):
        row['windy'] = 1

    if any(code in w_codes for code in rain):
        row['rain'] = 1

    if any(code in w_codes for code in lowvis):
        row['lowvis'] = 1

    if any(code in w_codes for code in codesum_others):
        row['codesum_others'] = 1

    return row

In [442]:
weather_df = weather_df.apply(w_code_dummy_split, axis = 1)

In [443]:
weather_df.head()

,AvgSpeed,CodeSum,Cool,Date,Depart,DewPoint,Heat,Normal,PrecipTotal,ResultDir,ResultSpeed,SeaLevel,Station,StnPressure,Sunrise,Sunset,Tavg,Tmax,Tmin,WetBulb,codesum_others,lowvis,n_weather_per_day,rain,snow,windy
0,9.2,,2,2007-05-01,14,51,0,53.0,0.00,27,1.7,29.82,1,29.10,0448,1849,67,83,50,56,NaN,NaN,0,NaN,NaN,NaN
1,9.6,,3,2007-05-01,15,51,0,53.0,0.00,25,2.7,29.82,2,29.18,0448,1849,68,84,52,57,NaN,NaN,0,NaN,NaN,NaN
2,13.4,BR,0,2007-05-02,-3,42,14,54.0,0.00,4,13.0,30.09,1,29.38,0447,1850,51,59,42,47,NaN,1.0,1,NaN,NaN,NaN
3,13.4,BR HZ,0,2007-05-02,-2,42,13,54.0,0.00,2,13.3,30.08,2,29.44,0447,1850,52,60,43,47,NaN,1.0,2,NaN,NaN,NaN
4,11.9,,0,2007-05-03,2,40,9,54.0,0.00,7,11.7,30.12,1,29.39,0446,1851,56,66,46,48,NaN,NaN,0,NaN,NaN,NaN


In [444]:
# Rearrange the columns
weather_df = weather_df[['Station',
 'Date',
 'Tmax',
 'Tmin',
 'Tavg',
 'Depart',
 'DewPoint',
 'WetBulb',
 'Heat',
 'Cool',
 'Normal',
 'Sunrise',
 'Sunset',
 'CodeSum',
 'PrecipTotal',
 'StnPressure',
 'SeaLevel',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed',
 'snow',
 'windy',
 'rain',
 'lowvis',
 'codesum_others']]

In [445]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Normal,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,snow,windy,rain,lowvis,codesum_others
0,1,2007-05-01,83,50,67,14,51,56,0,2,53.0,0448,1849,,0.00,29.10,29.82,1.7,27,9.2,NaN,NaN,NaN,NaN,NaN
1,2,2007-05-01,84,52,68,15,51,57,0,3,53.0,0448,1849,,0.00,29.18,29.82,2.7,25,9.6,NaN,NaN,NaN,NaN,NaN
2,1,2007-05-02,59,42,51,-3,42,47,14,0,54.0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4,NaN,NaN,NaN,1.0,NaN
3,2,2007-05-02,60,43,52,-2,42,47,13,0,54.0,0447,1850,BR HZ,0.00,29.44,30.08,13.3,2,13.4,NaN,NaN,NaN,1.0,NaN
4,1,2007-05-03,66,46,56,2,40,48,9,0,54.0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9,NaN,NaN,NaN,NaN,NaN


In [446]:
weather_df[['snow', 'windy', 'rain', 'lowvis', 'codesum_others']].sum()

snow                 6.0
windy                3.0
rain              1007.0
lowvis             882.0
codesum_others       9.0
dtype: float64

In [447]:
# Will be dropping snow, windy and others as  their occurence is very low

weather_df = weather_df.drop(columns= ['snow', 'windy', 'codesum_others'])

In [448]:
# Change all datatype to float

for col in weather_df.columns:
    try:
        weather_df[col] = weather_df[col].astype(float)
    except:
        print(col, 'cannot be transformed into float')
        pass

Date cannot be transformed into float
CodeSum cannot be transformed into float


In [449]:
# Seems about right

weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   float64       
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   float64       
 3   Tmin         2944 non-null   float64       
 4   Tavg         2944 non-null   float64       
 5   Depart       2944 non-null   float64       
 6   DewPoint     2944 non-null   float64       
 7   WetBulb      2944 non-null   float64       
 8   Heat         2944 non-null   float64       
 9   Cool         2944 non-null   float64       
 10  Normal       2944 non-null   float64       
 11  Sunrise      2944 non-null   float64       
 12  Sunset       2944 non-null   float64       
 13  CodeSum      2944 non-null   object        
 14  PrecipTotal  2944 non-null   float64       
 15  StnPressure  2944 non-null   float64       
 16  SeaLev

Merging Station 1 and Station 2 results to make our analysis easier

In [450]:
weather_df = weather_df.groupby('Date').sum() / 2
weather_df = weather_df.drop(columns = ['Station', 'Normal']).reset_index()

C:\Users\Admin\AppData\Local\Temp\ipykernel_1740\269615993.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather_df = weather_df.groupby('Date').sum() / 2


In [451]:
weather_df.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,rain,lowvis
0,2007-05-01,83.5,51.0,67.5,14.5,51.0,56.5,0.0,2.5,448.0,1849.0,0.000,29.140,29.820,2.20,26.0,9.40,0.0,0.0
1,2007-05-02,59.5,42.5,51.5,-2.5,42.0,47.0,13.5,0.0,447.0,1850.0,0.000,29.410,30.085,13.15,3.0,13.40,0.0,1.0
2,2007-05-03,66.5,47.0,57.0,3.0,40.0,49.0,8.0,0.0,446.0,1851.0,0.000,29.425,30.120,12.30,6.5,12.55,0.0,0.5
3,2007-05-04,72.0,50.0,36.0,-18.0,41.5,50.0,29.0,0.0,444.0,1852.0,0.005,29.335,30.045,10.25,7.5,10.60,0.5,0.0
4,2007-05-05,66.0,53.5,60.0,5.0,38.5,49.5,5.0,0.0,443.0,1853.0,0.010,29.430,30.095,11.45,7.0,11.75,0.0,0.0


In [452]:
# Add year, month , week and day of week features

weather_df['Year'] = weather_df['Date'].apply(lambda x: x.year)
weather_df['Month'] = weather_df['Date'].apply(lambda x: x.month)
weather_df['Week'] = weather_df['Date'].apply(lambda x: x.week)
weather_df['DayOfWeek'] = weather_df['Date'].apply(lambda x: x.dayofweek)

In [453]:
weather_df.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,rain,lowvis,Year,Month,Week,DayOfWeek
0,2007-05-01,83.5,51.0,67.5,14.5,51.0,56.5,0.0,2.5,448.0,1849.0,0.000,29.140,29.820,2.20,26.0,9.40,0.0,0.0,2007,5,18,1
1,2007-05-02,59.5,42.5,51.5,-2.5,42.0,47.0,13.5,0.0,447.0,1850.0,0.000,29.410,30.085,13.15,3.0,13.40,0.0,1.0,2007,5,18,2
2,2007-05-03,66.5,47.0,57.0,3.0,40.0,49.0,8.0,0.0,446.0,1851.0,0.000,29.425,30.120,12.30,6.5,12.55,0.0,0.5,2007,5,18,3
3,2007-05-04,72.0,50.0,36.0,-18.0,41.5,50.0,29.0,0.0,444.0,1852.0,0.005,29.335,30.045,10.25,7.5,10.60,0.5,0.0,2007,5,18,4
4,2007-05-05,66.0,53.5,60.0,5.0,38.5,49.5,5.0,0.0,443.0,1853.0,0.010,29.430,30.095,11.45,7.0,11.75,0.0,0.0,2007,5,18,5


In [454]:
# Export cleaned dataset

weather_df.to_csv('../data/cleaned_weather.csv', index = False)

***Train***

In [455]:
# Add Year, Month, Week and Day of Week features
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].apply(lambda x: x.year)
train_df['Month'] = train_df['Date'].apply(lambda x: x.month)
train_df['Week'] = train_df['Date'].apply(lambda x: x.week)
train_df['DayOfWeek'] = train_df['Date'].apply(lambda x: x.dayofweek)

In [456]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Block                   10506 non-null  int64         
 4   Street                  10506 non-null  object        
 5   Trap                    10506 non-null  object        
 6   AddressNumberAndStreet  10506 non-null  object        
 7   Latitude                10506 non-null  float64       
 8   Longitude               10506 non-null  float64       
 9   AddressAccuracy         10506 non-null  int64         
 10  NumMosquitos            10506 non-null  int64         
 11  WnvPresent              10506 non-null  int64         
 12  Year                    10506 non-null  int64 

In [457]:
# Check for missing values

weather_df.isnull().sum().sort_values(ascending= False)

Date           0
StnPressure    0
Week           0
Month          0
Year           0
lowvis         0
rain           0
AvgSpeed       0
ResultDir      0
ResultSpeed    0
SeaLevel       0
PrecipTotal    0
Tmax           0
Sunset         0
Sunrise        0
Cool           0
Heat           0
WetBulb        0
DewPoint       0
Depart         0
Tavg           0
Tmin           0
DayOfWeek      0
dtype: int64

In [458]:
train_df.isnull().sum()[train_df.isnull().sum() > 0]

Series([], dtype: int64)

In [459]:
# Checking for duplicated values

train_df.duplicated().sum()

813

In [460]:
train_df.duplicated().sum()[train_df.duplicated().sum() > 0]

array([813], dtype=int64)

In [461]:
duplicates = train_df[train_df.duplicated()]
print(duplicates)

            Date                                            Address  \
99    2007-06-26     2200 West 113th Street, Chicago, IL 60643, USA   
295   2007-07-11     2200 West 113th Street, Chicago, IL 60643, USA   
351   2007-07-11     3500 West 116th Street, Chicago, IL 60655, USA   
353   2007-07-11     3500 West 116th Street, Chicago, IL 60655, USA   
512   2007-07-18  3300 West Randolph Street, Chicago, IL 60612, USA   
531   2007-07-18        South Stony Island Avenue, Chicago, IL, USA   
548   2007-07-18          3700 118th Street, Chicago, IL 60617, USA   
587   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
588   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
589   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
590   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
591   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
593   2007-07-19          3700 118th Street, Chicago, IL 60617, USA   
602   

In [462]:
train_df.iloc[[548, 587, 588, 589, 590, 591, 593]]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Week,DayOfWeek
548,2007-07-18,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,2
587,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3
588,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3
589,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3
590,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3
591,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3
593,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,29,3


There are some entries among the duplicated rows that indicated there is WnvPresent, whereas the other duplicate copies does not, as such, will keep them in the dataset just in case

In [463]:
# Just to check number of cases of Wnv occurences

a = train_df[train_df['WnvPresent'] != 0]


In [464]:
# 551 occurences
a.shape

(551, 16)

In [465]:
# Nothing else much to do in this dataset, considered clean and export it

train_df.to_csv('../data/cleaned_train.csv')

***Test***

In [466]:
# Add Year, Month, Week and Day of Week features
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df['Year'] = test_df['Date'].apply(lambda x: x.year)
test_df['Month'] = test_df['Date'].apply(lambda x: x.month)
test_df['Week'] = test_df['Date'].apply(lambda x: x.week)
test_df['DayOfWeek'] = test_df['Date'].apply(lambda x: x.dayofweek)

In [467]:
# Otherwise, nothing else much to cleaning work needs to be done to this dataset as well, export it

test_df.to_csv('../data/cleaned_test.csv')

***Spray***

In [472]:
# There are some null values in the time feature

spray_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [479]:
spray_df.loc[spray_df.Time.isnull()]

,Date,Time,Latitude,Longitude
1030,2011-09-07,NaN,41.987092,-87.794286
1031,2011-09-07,NaN,41.987620,-87.794382
1032,2011-09-07,NaN,41.988004,-87.794574
1033,2011-09-07,NaN,41.988292,-87.795486
1034,2011-09-07,NaN,41.988100,-87.796014
1035,2011-09-07,NaN,41.986372,-87.794862
1036,2011-09-07,NaN,41.986228,-87.795582
1037,2011-09-07,NaN,41.984836,-87.793998
1038,2011-09-07,NaN,41.984836,-87.794670
1039,2011-09-07,NaN,41.984884,-87.795198


In [484]:
null_val = spray_df[spray_df['Time'].isnull()]
print(null_val)

            Date Time   Latitude  Longitude
1030  2011-09-07  NaN  41.987092 -87.794286
1031  2011-09-07  NaN  41.987620 -87.794382
1032  2011-09-07  NaN  41.988004 -87.794574
1033  2011-09-07  NaN  41.988292 -87.795486
1034  2011-09-07  NaN  41.988100 -87.796014
1035  2011-09-07  NaN  41.986372 -87.794862
1036  2011-09-07  NaN  41.986228 -87.795582
1037  2011-09-07  NaN  41.984836 -87.793998
1038  2011-09-07  NaN  41.984836 -87.794670
1039  2011-09-07  NaN  41.984884 -87.795198
1040  2011-09-07  NaN  41.982820 -87.793182
1041  2011-09-07  NaN  41.982772 -87.793950
1042  2011-09-07  NaN  41.982772 -87.795006
1043  2011-09-07  NaN  41.983444 -87.795246
1044  2011-09-07  NaN  41.984068 -87.795438
1045  2011-09-07  NaN  41.984596 -87.795534
1046  2011-09-07  NaN  41.984980 -87.795630
1047  2011-09-07  NaN  41.986708 -87.796110
1048  2011-09-07  NaN  41.987524 -87.796302
1049  2011-09-07  NaN  41.988820 -87.796638
1050  2011-09-07  NaN  41.989444 -87.796830
1051  2011-09-07  NaN  41.990068

In [487]:
spray_df.Time.value_counts().head()

7:44:32 PM    541
8:57:46 PM      5
9:31:27 PM      5
9:35:47 PM      5
9:37:27 PM      5
Name: Time, dtype: int64

In [488]:
spray_df.duplicated().sum()[spray_df.duplicated().sum() > 0]

array([541], dtype=int64)

In [491]:
duplicates = spray_df[spray_df.duplicated()]
print(duplicates)
duplicates.shape

            Date        Time   Latitude  Longitude
485   2011-09-07  7:43:40 PM  41.983917 -87.793088
490   2011-09-07  7:44:32 PM  41.986460 -87.794225
491   2011-09-07  7:44:32 PM  41.986460 -87.794225
492   2011-09-07  7:44:32 PM  41.986460 -87.794225
493   2011-09-07  7:44:32 PM  41.986460 -87.794225
494   2011-09-07  7:44:32 PM  41.986460 -87.794225
495   2011-09-07  7:44:32 PM  41.986460 -87.794225
496   2011-09-07  7:44:32 PM  41.986460 -87.794225
497   2011-09-07  7:44:32 PM  41.986460 -87.794225
498   2011-09-07  7:44:32 PM  41.986460 -87.794225
499   2011-09-07  7:44:32 PM  41.986460 -87.794225
500   2011-09-07  7:44:32 PM  41.986460 -87.794225
501   2011-09-07  7:44:32 PM  41.986460 -87.794225
502   2011-09-07  7:44:32 PM  41.986460 -87.794225
503   2011-09-07  7:44:32 PM  41.986460 -87.794225
504   2011-09-07  7:44:32 PM  41.986460 -87.794225
505   2011-09-07  7:44:32 PM  41.986460 -87.794225
506   2011-09-07  7:44:32 PM  41.986460 -87.794225
507   2011-09-07  7:44:32 PM  4

(541, 4)